In [114]:
import pandas as pd
import numpy as np
from openai import OpenAI
import os
import json

import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv('env.txt')

In [29]:
!pip install reportlab

     ---------------------------------------- 1.9/1.9 MB 1.2 MB/s eta 0:00:00
     -------------------------------------- 199.4/199.4 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 1.3 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# В файле env.txt содержутся ключи к API нейронок

os.environ['AI_TUNNEL_API'] = os.getenv('AI_TUNNEL_API')
os.environ['GIGA_CHAT_PRO'] = os.getenv('GIGA_CHAT_PRO')

## Data preprocessing

In [3]:
data_description = pd.read_parquet('parquet data\data_description.parquet', engine='pyarrow')
ed_izm = pd.read_parquet('parquet data\ed_izm.parquet', engine='pyarrow')
gost = pd.read_parquet('parquet data\gost.parquet', engine='pyarrow')
okpd2 = pd.read_parquet('parquet data\okpd2.parquet', engine='pyarrow')

In [4]:
# Редактирование основного файла

data_description.rename(columns={
    'код СКМТР': 'id',
    'Наименование': 'name',
    'Маркировка': 'mark_name',
    'Регламенты (ГОСТ/ТУ)': 'gost',
    'Параметры': 'params',
    'Базисная Единица измерения': 'bei',
    'ОКПД2': 'okpd'
}, inplace=True)

In [5]:
print(data_description.shape)
data_description.head()

(703291, 7)


,id,name,mark_name,gost,params,bei,okpd
0,8559150569,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-182 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131
1,8559150570,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-188 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131
2,8559150571,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-194 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131
3,8559150572,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,128-112-152 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131
4,8559150573,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,128-112-158 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131


In [7]:
# Редактирование файла с измерениями

replace_dict = {
    'A': 'А',  # латинская A на русскую А
    'B': 'В',  # латинская B на русскую В
    'E': 'Е',  # латинская E на русскую Е
    'K': 'К',  # латинская K на русскую К
    'M': 'М',  # латинская M на русскую М
    'H': 'Н',  # латинская H на русскую Н
    'O': 'О',  # латинская O на русскую О
    'P': 'Р',  # латинская P на русскую Р
    'C': 'С',  # латинская C на русскую С
    'T': 'Т',  # латинская T на русскую Т
    'X': 'Х',  # латинская X на русскую Х
}

# Функция для замены
def replace_eng_to_rus(text):
    original_text = text
    replacement_made = False  # Флаг для отслеживания замен

    for eng, rus in replace_dict.items():
        if eng in text:  # Если найден символ для замены
            text = text.replace(eng, rus)
            replacement_made = True  # Фиксируем, что замена была

    # Если замена произошла, выводим сообщение
    if replacement_made:
        print(f"Заменены английские буквы в строке: '{original_text}' на '{text}'")

    return text

# Применяем замену к столбцу датафрейма
ed_izm['Краткое'] = ed_izm['Краткое'].apply(replace_eng_to_rus)

In [8]:
# Редактирование файла с ГОСТами

gost = gost[['GOST#Gost_code', 'GOST#Gost_title']]
gost.rename(columns={'GOST#Gost_code':'gost', 'GOST#Gost_title':'gost_title'}, inplace=True)

In [9]:
# Редактирование файла с okpd

okpd2['OKPD2_NAME'] = okpd2['OKPD2_NAME'].str.split('Эта группировка ВКЛЮЧАЕТ').str[0].str.strip()

# Функция для восстановления предшествующих кодов
def restore_hierarchy(code):
    # Разбиваем код по точкам
    parts = code.split('.')
    # Генерируем предшествующие коды
    hierarchy = ['.'.join(parts[:i]) for i in range(1, len(parts)+1)]
    # Дополняем список кодов до максимального количества столбцов (например, 5 уровней)
    return hierarchy + [None] * (5 - len(hierarchy))  # 5 - максимальное количество уровней, можно изменить

# Применяем функцию к столбцу 'code' и создаем новые столбцы с предшествующими уровнями
hierarchy_df = okpd2['OKPD2'].apply(restore_hierarchy)
hierarchy_df = pd.DataFrame(hierarchy_df.tolist(), columns=[f'okpd2_level_{i}' for i in range(1, 6)])
okpd2 = pd.concat([okpd2, hierarchy_df], axis=1)

okpd2.drop(['okpd2_level_5'],axis=1, inplace=True)

In [10]:
data_description

,id,name,mark_name,gost,params,bei,okpd
0,8559150569,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-182 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131
1,8559150570,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-188 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131
2,8559150571,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-194 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131
3,8559150572,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,128-112-152 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131
4,8559150573,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,128-112-158 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131
...,...,...,...,...,...,...,...
703286,8559245745,ЖИЛЕТ ЖЕНСКИЙ ФОРМЕННЫЙ,ТЕХНОАВИА BUSINESS CLASS 1.124,ГОСТ 25295-2003,108/158 ШЕРСТЬ-44% ПОЛИЭФИР-54% ЭЛАСТАН-2% ТЕМ...,796,14.12.30.132
703287,8559245744,ЖИЛЕТ ЖЕНСКИЙ ФОРМЕННЫЙ,ТЕХНОАВИА BUSINESS CLASS 1.124,ГОСТ 25295-2003,104/176 ШЕРСТЬ-44% ПОЛИЭФИР-54% ЭЛАСТАН-2% ТЕМ...,796,14.12.30.132
703288,8559245743,ЖИЛЕТ ЖЕНСКИЙ ФОРМЕННЫЙ,ТЕХНОАВИА BUSINESS CLASS 1.124,ГОСТ 25295-2003,104/170 ШЕРСТЬ-44% ПОЛИЭФИР-54% ЭЛАСТАН-2% ТЕМ...,796,14.12.30.132
703289,8559245742,ЖИЛЕТ ЖЕНСКИЙ ФОРМЕННЫЙ,ТЕХНОАВИА BUSINESS CLASS 1.124,ГОСТ 25295-2003,104/164 ШЕРСТЬ-44% ПОЛИЭФИР-54% ЭЛАСТАН-2% ТЕМ...,796,14.12.30.132


In [11]:
gost

,gost,gost_title
0,ГОСТ Р 71203-2024,Напитки виноградные крепкие. Общие технические...
1,ГОСТ Р 113.00.04-2024,Наилучшие доступные технологии. Формат описани...
2,ГОСТ 28357-2023,Продукты коксохимические. Ускоренный метод опр...
3,ГОСТ Р 71209-2024,Станки электрохимические копировально-прошивоч...
4,ГОСТ 28572-2023,Пек каменноугольный. Диэлектрический метод опр...
...,...,...
50085,ГОСТ 3478-79,Подшипники качения. Основные размеры
50086,ГОСТ 17649-72,Консервы. Фасоль или горох со шпиком или свины...
50087,ГОСТ 13340.1-77,"Овощи сушеные. Методы определения массы нетто,..."
50088,ГОСТ 17614-80,Теллур технический. Технические условия


In [12]:
main_df = data_description.merge(okpd2, how='left', left_on='okpd', right_on='OKPD2')
main_df = main_df.merge(gost, how='left', on='gost')
main_df.drop(['OKPD2'], axis=1, inplace=True)

In [114]:
main_df.to_parquet('main_df.parquet')

In [31]:
main_df

,id,name,mark_name,gost,params,bei,okpd,OKPD2_NAME,okpd2_level_1,okpd2_level_2,okpd2_level_3,okpd2_level_4,gost_title
0,8559150569,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-182 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131,"Халаты, фартуки, жилеты и сорочки мужские прои...",14,14.12,14.12.30,14.12.30.131,NaN
1,8559150570,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-188 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131,"Халаты, фартуки, жилеты и сорочки мужские прои...",14,14.12,14.12.30,14.12.30.131,NaN
2,8559150571,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-194 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131,"Халаты, фартуки, жилеты и сорочки мужские прои...",14,14.12,14.12.30,14.12.30.131,NaN
3,8559150572,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,128-112-152 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131,"Халаты, фартуки, жилеты и сорочки мужские прои...",14,14.12,14.12.30,14.12.30.131,NaN
4,8559150573,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,128-112-158 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131,"Халаты, фартуки, жилеты и сорочки мужские прои...",14,14.12,14.12.30,14.12.30.131,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
703286,8559245745,ЖИЛЕТ ЖЕНСКИЙ ФОРМЕННЫЙ,ТЕХНОАВИА BUSINESS CLASS 1.124,ГОСТ 25295-2003,108/158 ШЕРСТЬ-44% ПОЛИЭФИР-54% ЭЛАСТАН-2% ТЕМ...,796,14.12.30.132,"Юбки, халаты, блузки, фартуки, жилеты, платья ...",14,14.12,14.12.30,14.12.30.132,Одежда верхняя пальтово-костюмного ассортимент...
703287,8559245744,ЖИЛЕТ ЖЕНСКИЙ ФОРМЕННЫЙ,ТЕХНОАВИА BUSINESS CLASS 1.124,ГОСТ 25295-2003,104/176 ШЕРСТЬ-44% ПОЛИЭФИР-54% ЭЛАСТАН-2% ТЕМ...,796,14.12.30.132,"Юбки, халаты, блузки, фартуки, жилеты, платья ...",14,14.12,14.12.30,14.12.30.132,Одежда верхняя пальтово-костюмного ассортимент...
703288,8559245743,ЖИЛЕТ ЖЕНСКИЙ ФОРМЕННЫЙ,ТЕХНОАВИА BUSINESS CLASS 1.124,ГОСТ 25295-2003,104/170 ШЕРСТЬ-44% ПОЛИЭФИР-54% ЭЛАСТАН-2% ТЕМ...,796,14.12.30.132,"Юбки, халаты, блузки, фартуки, жилеты, платья ...",14,14.12,14.12.30,14.12.30.132,Одежда верхняя пальтово-костюмного ассортимент...
703289,8559245742,ЖИЛЕТ ЖЕНСКИЙ ФОРМЕННЫЙ,ТЕХНОАВИА BUSINESS CLASS 1.124,ГОСТ 25295-2003,104/164 ШЕРСТЬ-44% ПОЛИЭФИР-54% ЭЛАСТАН-2% ТЕМ...,796,14.12.30.132,"Юбки, халаты, блузки, фартуки, жилеты, платья ...",14,14.12,14.12.30,14.12.30.132,Одежда верхняя пальтово-костюмного ассортимент...


In [14]:
openai_key = os.environ.get('AI_TUNNEL_API')

In [15]:
client = OpenAI(
    api_key=openai_key,
    base_url="https://api.aitunnel.ru/v1/",
)

In [85]:
test = main_df[main_df['okpd'] == '14.12.30.131'].sample(100)

In [86]:
okpd_name = test['OKPD2_NAME'].unique()[0]

In [87]:
test['params']

478051    92-76-188 БЕЛАЯ В КОРИЧНЕВО-КРАСНУЮ ПОЛОСКУ (2...
404820    120-98-158 БЕЛЫЙ (1-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯ...
487248    140-118-170 БЕЛАЯ В КОРИЧНЕВО-КРАСНУЮ ПОЛОСКУ ...
382001                                              148-170
486523    160-138-194 БЕЛАЯ В КОРИЧНЕВО-КРАСНУЮ ПОЛОСКУ ...
                                ...                        
284443    112-102-182 (3-Я ПОЛНОТНАЯ ГРУППА) РЯДОВАЯ КАТ...
436109                                     124-68-188 БЕЛЫЙ
310773                                      144-148 158-164
525709    96-74-170 БЕЛЫЙ (1-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ...
445264    80-70-158 БЕЛЫЙ В СЕРУЮ ПОЛОСКУ (3-АЯ ПОЛНОТНА...
Name: params, Length: 100, dtype: object

In [88]:
context = ";".join(ed_izm['Наименование'] + '->' + ed_izm['Краткое'])

In [89]:
data_description[data_description['params'].isna()]

,id,name,mark_name,gost,params,bei,okpd
23,5775310030,МАСТИКА БИТУМНО-РЕЗИНОВАЯ ХОЛОДНАЯ,МБР-Х-65,ТУ 23.99.12-015-00287823-2010,None,166,23.99.12.120
118,4531180186,КОРОБКА РАЗДАТОЧНАЯ,4320ЯМ-1800020-10,None,None,796,29.32.30.390
132,3187865609,КРЫШКА,ДГКУЖ5.02.00.017,None,None,796,30.20.40.149
318,4981700033,ЗАМОК ЗАЩЕЛКА ДЛЯ ПРОФЕЛЬНЫХ ДВЕРЕЙ ЛЕВЫЙ,ABLOY LC303/35L,None,None,796,25.72.14.120
443,6673101719,СМАРТФОН,ACER BETOUCH E400,None,None,796,26.30.22.110
...,...,...,...,...,...,...,...
703077,9522000052,ЖУРНАЛ ПО ПОДПИСКЕ,ВАГОНЫ И ВАГОННОЕ ХОЗЯЙСТВО ПРИЛОЖЕНИЕ К ЖУРНА...,None,None,839,58.14.12.000
703087,9523100549,ЖУРНАЛ ПО ПОДПИСКЕ,ЖЕЛЕЗНОДОРОЖНЫЙ ТРАНСПОРТ,None,None,839,58.14.12.000
703088,9527100039,ЖУРНАЛ ПО ПОДПИСКЕ,ЖЕЛЕЗНЫЕ ДОРОГИ МИРА,None,None,839,58.14.11.190
703097,5611200003,СТОЛ ДВУХТУМБОВЫЙ,ОН-11-386/9,МРТУ 13-08-01-65,None,796,31.09.12.139


In [90]:
params = '[' + "]; [".join(test[~test['params'].isna()]['params']) + ']'

In [91]:
len(params)

7602

In [92]:
len(f"""Ты рассматриваешь класс продуктов "{okpd_name}". У каждого продукта в этом классе есть определенные параметры, характерные для этого типа продуктов. Вот эти параметры: <{params}>. Каждый объект заключенный в '[]' - параметры отдельного продукта, с id равным номеру этого продукта в общем массиве. На основе переданного словаря обозначений параметров ({context}) тебе необходимо восстановить полные названия этих параметров и сформировать ответ, содержащий топ-10 самых частых из них для кажого продукта. Вот шаблон итогового ответа:
    <<<Параметр1: [Значение для продукта с id = 1, Значение для продукта с id = 2, ...];
    Параметр2: [Значение для продукта с id = 1, Значение для продукта с id = 2, ...];
    ...>>>""")

20427

In [97]:
completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Ты — полезный ассистент, который отвечает кратко и интересно."},
        {"role": "user", "content": f"""Ты рассматриваешь класс продуктов "{okpd_name}". У каждого продукта в этом классе есть определенные параметры, характерные для этого типа продуктов. Вот эти параметры: <{params}>. Каждый объект заключенный в '[]' - параметры отдельного продукта, с id равным номеру этого продукта в общем массиве. На основе переданного словаря обозначений параметров ({context}) тебе необходимо восстановить полные названия этих параметров и сформировать ответ по следующему шаблону:
    <<<Параметр1: ['Значение для продукта с id = 1', 'Значение для продукта с id = 2', ...];
    Параметр2: ['Значение для продукта с id = 1', 'Значение для продукта с id = 2', ...];
    ...>>>"""}
    ],
    max_tokens=20000,
    model="gpt-4o-mini"
)

In [98]:
print(completion.choices[0].message.content)

<<<Размер: ['92-76-188', '120-98-158', '140-118-170', '148-170', '160-138-194', '96-80-188', '128-106-200', '140-142-200', '144-146-152', '124-126-182', '120-122-200', '84-96-170', '96-74-194', '92-82-182', '88-66-152', '132-76-200', '84-80-176', '136-120-188', '152-142-176', '160-138-182', '128-104-176', '128-124-200', '140-136-152', '112-104-170', '200-88-90-38', '100-84-182', '144-194', '104-94-194', '104-82-176', '140-118-170', '96-164', '152-154-164', '164-112-96-44', '108-92-164', '120-188', '124-114-194', '136-114-164', '132-128-182', '148-138-176', '124-114-170', '128-112-182', '104-100-194', '88-90-164', '152-148-182', '144-138-164', '140-136-170', '100-102-182', '144-146-176', 'АССОРТИ', '132-122-188', '144-148 170-176', '96-98-200', '96-98-176', '152-154-188', '88-92 146-152', '128-112-152', '104-82-176', '96-92-176', '88-96-170', '92-92-158', '152-156 170-176', '124-114-194', '112-90-170', '116-118-170', '100-102-176', '160-156-176', '156-146-182', '144-122-170', '124-120-1

In [172]:
with open('govno_v4.json', 'r', encoding='utf-8') as json_file:
    wanna_cry_pro = json.load(json_file)

In [180]:
wanna_cry_pro = json.loads(wanna_cry_pro.replace('json', '').replace('```', '').replace('\n', '').replace("'", '"'))
wanna_cry_pro

[{'ID': 253023, 'Размеры': '92-94-176', 'Полнотная группа': '5-я'},
 {'ID': 248993, 'Размеры': '140-142-152', 'Полнотная группа': '5-я'},
 {'ID': 248290, 'Размеры': '148-138-194', 'Полнотная группа': '3-я'},
 {'ID': 248356, 'Размеры': '112-102-158', 'Полнотная группа': '3-я'},
 {'ID': 247641, 'Размеры': '112-90-164', 'Полнотная группа': '1-я'},
 {'ID': 252878, 'Размеры': '112-102-170', 'Полнотная группа': '3-я'},
 {'ID': 248998, 'Размеры': '144-146-158', 'Полнотная группа': '5-я'},
 {'ID': 252898, 'Размеры': '124-114-182', 'Полнотная группа': '3-я'},
 {'ID': 289392, 'Размеры': '96-100 170-176', 'Цвет': 'Тёмно-синий'},
 {'ID': 247824, 'Размеры': '100-84-152', 'Полнотная группа': '2-я'},
 {'ID': 246690, 'Размеры': '148-126-158', 'Полнотная группа': '1-я'},
 {'ID': 426741, 'Размеры': '128-108-188', 'Цвет': 'Синий'},
 {'ID': 425998, 'Размеры': '100-100-176', 'Цвет': 'Синий'},
 {'ID': 425575, 'Размеры': '84-72-176', 'Цвет': 'Синий'},
 {'ID': 248933, 'Размеры': '116-118-170', 'Полнотная груп

In [181]:
with open('output)2.json', 'w', encoding='utf-8') as json_file:
    json.dump(wanna_cry_pro, json_file, ensure_ascii=False, indent=4)

In [168]:
result_dict = {}
for key in wanna_cry.keys():
    result_list = []
    for data in wanna_cry[key]:
        jsn = data.replace('json', '').replace('```', '').replace("'", '"').replace('/n', '')
        jsn = json.loads(jsn)
        result_list.append(jsn)
    result_dict[key] = result_list

In [170]:
with open('output.json', 'w', encoding='utf-8') as json_file:
    json.dump(result_dict, json_file, ensure_ascii=False, indent=4)

In [117]:
data['14.12.30.132']

TypeError: string indices must be integers